In [1]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.utils.constants import ES_THRESHOLD, METAL_OX_STATES
from tqdm import tqdm 

In [2]:
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol, get_vdw_radius, add_charges_to_molecule, get_node_features, get_edge_features, extract_charges_from_mol2, ionic_interaction_using_distance

In [3]:
def calculate_gasteiger_charges(mol):
    AllChem.ComputeGasteigerCharges(mol)
    for atom in mol.GetAtoms():
        atom.SetProp('_TriposAtomCharges', str(atom.GetDoubleProp('_GasteigerCharge')))
    return mol


In [4]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  (0, 0, 0, 0, 0, 1.3261131312034182)
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0, 2.278544145756904)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    mol = calculate_gasteiger_charges(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    result = ionic_interaction_using_distance(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Result: {result}")


SMILES: C=CCCCCCCCCC, Atoms: (0, 10) -> Result: None
SMILES: C=C, Atoms: (0, 1) -> Result: (0, 0, 0, 0, 0, 1.327022705645877)
SMILES: CC(=O)O, Atoms: (2, 3) -> Result: (0, 0, 0, 0, 0, 2.2763370871071418)


[21:34:29] Molecule does not have explicit Hs. Consider calling AddHs()
[21:34:30] Molecule does not have explicit Hs. Consider calling AddHs()
[21:34:30] Molecule does not have explicit Hs. Consider calling AddHs()
